In [1]:
import sqlite3
import pandas as pd

In [2]:
# Establish Database

conn = sqlite3.connect('data/eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [3]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


In [4]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [5]:
df.shape

(82498, 3)

In [8]:
# here 2 represent the index of content column
# 0 represents the row number

b_data = df.loc[0, 2]
# print(b_data)

In [9]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [10]:
df['file_content'] = df['content'].apply(decode_method)

df.head()

,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           82498 non-null  int64 
 1   name          82498 non-null  object
 2   content       82498 non-null  object
 3   file_content  82498 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.5+ MB


In [12]:
df['file_content'][0][0:1000]

'1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch any video online with Open-SUBTITLES\r\nFree Browser extension: osdb.link/ext\r\n\r\n2\r\n00:02:26,198 --> 00:02:29,953\r\nIn the name of God, the most gracious, the most Merciful.\r\n\r\n3\r\n00:02:31,072 --> 00:02:33,370\r\nFrom Muhammad, the Messenger of God\r\n\r\n4\r\n00:02:33,550 --> 00:02:36,047\r\nto Heraclius, the emperor of Byzantium.\r\n\r\n5\r\n00:02:36,407 --> 00:02:39,464\r\ngreetings to him who is the\r\nfollower of righteous guidance.\r\n\r\n6\r\n00:02:39,783 --> 00:02:42,591\r\nI bid you to hear the divine call.\r\n\r\n7\r\n00:02:43,160 --> 00:02:45,817\r\nI am the messenger of God to the people;\r\n\r\n8\r\n00:02:46,337 --> 00:02:48,784\r\naccept Islam for your salvation.\r\n\r\n9\r\n00:02:52,231 --> 00:02:54,709\r\nHe speaks of a new prophet in Arabia.\r\n\r\n10\r\n00:02:55,068 --> 00:02:57,825\r\nWas it like this when John, the Baptist\r\ncame to king Herod\r\n\r\n11\r\n00:02:58,145 --> 00:03:01,272\r\nout of the desert, 

In [13]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def clean(doc): 
    # doc is a string of text
    
    # Let's define a regex to match special characters and digits
    regex = "[^a-zA-Z.]"
    doc = re.sub(regex, " ", doc)

    # Convert to lowercase
    doc = doc.lower()
        
    # Tokenization
    tokens = nltk.word_tokenize(doc)

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join and return
    return " ".join(lemmatized_tokens)

In [14]:
from tqdm import tqdm

tqdm.pandas()

df['subtitles'] = df['file_content'].progress_apply(lambda x: clean(x))

df.head()

100%|██████████| 82498/82498 [34:34<00:00, 39.77it/s]  


,num,name,content,file_content,subtitles
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with open subtitle free...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...",ah there s princess dawn and terry with the bl...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...",i yumi s cell i i episode extremely polite yum...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...",watch any video online with open subtitle free...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...",watch any video online with open subtitle free...


In [15]:
print(df['subtitles'][0][0:1000])

watch any video online with open subtitle free browser extension osdb.link ext in the name of god the most gracious the most merciful . from muhammad the messenger of god to heraclius the emperor of byzantium . greeting to him who is the follower of righteous guidance . i bid you to hear the divine call . i am the messenger of god to the people accept islam for your salvation . he speaks of a new prophet in arabia . wa it like this when john the baptist came to king herod out of the desert cry about salvation to muqawqis patriarch of alexandria . kisra emperor of persia . muhammad call you with the call of god . accept islam for your salvation ... embrace islam . you come out of the desert smelling of camel and goat . to tell persia where he should kneel muhammad messenger of god . who gave him this authority god sent muhammad a a mercy to mankind . the scholar and historian of islam the university of al azhar in cairo the high islamic congress of the shiat in lebanon the maker of this

In [16]:
print('Length of document', len(df['subtitles'][0]))
print('number of documents', len(df))

Length of document 47123
number of documents 82498


In [17]:
import re

def extract_metadata(name):
    # First check if the name contains season and episode information (series)
    series_match = re.match(r"([a-zA-Z\s\.]+)\.s\d{2}\.e\d{2}", name)
    
    if series_match:
        # For series: Extract name and replace dots with spaces
        extracted_name = series_match.group(1).strip().replace('.', ' ')
        
        # Extract season, episode, and year
        season_match = re.search(r"\.s(\d+)", name)
        season = season_match.group(1) if season_match else None
        
        episode_match = re.search(r"\.e(\d+)", name)
        episode = episode_match.group(1) if episode_match else None
        
        year_match = re.search(r"\((\d{4})\)", name)
        year = year_match.group(1) if year_match else None
        
    else:
        # For movies: Extract name before the year (if no season and episode)
        extracted_name = re.match(r"([a-zA-Z\s\.]+)\.\(\d{4}\)", name).group(1).strip().replace('.', ' ') if re.match(r"([a-zA-Z\s\.]+)\.\(\d{4}\)", name) else name
        
        # Movie name doesn't have season or episode, so only extract the year
        season, episode = None, None
        
        year_match = re.search(r"\((\d{4})\)", name)
        year = year_match.group(1) if year_match else None
    
    return extracted_name, season, episode, year

In [19]:
# Apply the function to the 'name' column

df[['extracted_name', 'season', 'episode', 'year']] = df['name'].apply(lambda x: pd.Series(extract_metadata(x)))

In [20]:
df = df[['num', 'extracted_name', 'season', 'episode', 'year', 'subtitles']]

df.head(3)

,num,extracted_name,season,episode,year,subtitles
0,9180533,the message,None,None,1976,watch any video online with open subtitle free...
1,9180583,here comes the grump,01,09,1969,ah there s princess dawn and terry with the bl...
2,9180592,yumis cells,02,13,2022,i yumi s cell i i episode extremely polite yum...


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   num             82498 non-null  int64 
 1   extracted_name  82498 non-null  object
 2   season          61829 non-null  object
 3   episode         61829 non-null  object
 4   year            80446 non-null  object
 5   subtitles       82498 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.8+ MB


In [22]:
df['season'] = df['season'].fillna('movie')
df['episode'] = df['episode'].fillna('movie')
df['year'] = df['year'].fillna('unknown')

df.rename(columns={'extracted_name': 'name'}, inplace=True)
df.rename(columns={'num': 'movie_id'}, inplace=True)

In [23]:
df.head()

,movie_id,name,season,episode,year,subtitles
0,9180533,the message,movie,movie,1976,watch any video online with open subtitle free...
1,9180583,here comes the grump,01,09,1969,ah there s princess dawn and terry with the bl...
2,9180592,yumis cells,02,13,2022,i yumi s cell i i episode extremely polite yum...
3,9180594,yumis cells,02,14,2022,watch any video online with open subtitle free...
4,9180600,broker,movie,movie,2022,watch any video online with open subtitle free...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movie_id   82498 non-null  int64 
 1   name       82498 non-null  object
 2   season     82498 non-null  object
 3   episode    82498 non-null  object
 4   year       82498 non-null  object
 5   subtitles  82498 non-null  object
dtypes: int64(1), object(5)
memory usage: 3.8+ MB


In [25]:
df.to_csv('data/subtitles_metadata.csv', index=False)